# Prelim Exploration

Wayne H Nixalo

Kaggle Statoil Iceberg Satellite Imagery Classification Challenge

21 Jan 2018

2 days to go.

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [6]:
import json

In [76]:
from fastai.conv_learner import ConvLearner
from fastai.imports import *
from fastai.dataset import *

Which architecture to use?

In [4]:
from fastai.model import resnet34

In [11]:
with open('data/train.json') as jfile:
    train_data = json.load(jfile)

In [19]:
train_data[0].keys()

dict_keys(['id', 'band_1', 'band_2', 'inc_angle', 'is_iceberg'])

In [22]:
def array_to_np_band(data_array, key=None):
    """Returns an (n,75,75,1) ndarray from Statoil radar bands"""
    if key==None:
        print("Error: Key not specified")
        return
    return np.expand_dims(np.array([np.array(band[key]).astype(np.float32).reshape(75,75) for 
                     band in data_array]), axis=-1)

In [35]:
train_band_1 = array_to_np_band(train_data, 'band_1')
train_band_2 = array_to_np_band(train_data, 'band_2')
train_band_z = np.zeros(train_band_1.shape).astype(np.float32)

In [31]:
# NAs = 0
# for i in range(len(train_data)):
# #     print(train_data[i]['inc_angle'])
#     if type(train_data[i]['inc_angle']) != float:
#         print(type(train_data[i]['inc_angle']))
#         print(i)
#         break

<class 'str'>
101


In [27]:
# type(train_data[0]['inc_angle'])

float

In [33]:
# train_data[101]['inc_angle']

'na'

In [36]:
train_band_1.shape

(1604, 75, 75, 1)

In [40]:
np.concatenate((train_band_1, train_band_2, train_band_z), axis=-1).shape

(1604, 75, 75, 3)

In [41]:
len(train_data[0]['band_1'])

5625

In [42]:
75*75

5625

In [46]:
band = train_data[0]
key = 'band_1'

# band = np.expand_dims(np.array([np.array(band[key]).astype(np.float32).reshape(75,75) for 
#                      band in data_array]), axis=-1)

np.expand_dims(np.array([np.array(train_data[0]['band_1']).astype(
                                        np.float32).reshape(75,75)]), axis=-1).shape

(1, 75, 75, 1)

In [49]:
np.array([np.array(train_data[0]['band_1']).astype(np.float32).reshape(75,75)]).shape

(1, 75, 75)

The `np.expand_dims` adds the dimension to the end, specified by `axis=-1`. This allows concetenation later on to have a compatible 3-channel input to the pretrained ConvNets.

`np.array([....])` is adding a dimension to the start. When run via list-comprehension on the entire dataset, this allows creating an `N`-long array of datums.

In [51]:
75*75*3

16875

What kind of data are the bands? Do I need to convert to 32-bit Floats?

In [55]:
type(train_data[0]['band_1'][0])

False

Python3 floats are 53-bit? [link](https://docs.python.org/3/tutorial/floatingpoint.html#15.1)

Anyway that's a lot more than 32.

In [65]:
bin(2**32-1)

'0b11111111111111111111111111111111'

In [68]:
train_data[0]['band_1'][:5]

[-27.878361, -27.15416, -28.668615, -29.537971, -29.092485]

I can easily fit 2 leading decimal digits, and 6 trailing digits in a 32-bit float yeah? 32-bit is 9 dec digits. 

In [74]:
train_band_1 = array_to_np_band(train_data, 'band_1')
train_band_2 = array_to_np_band(train_data, 'band_2')
train_band_z = np.zeros(train_band_1.shape).astype(np.float32)
train_bands  = np.concatenate([train_band_1,train_band_2,train_band_z],axis=-1)

In [75]:
train_bands.shape

(1604, 75, 75, 3)

In [87]:
n = train_bands.shape[0]
val_idxs = get_cv_idxs(n)

In [ ]:
train_data_array = (train_bands)
val_data_array = 

In [89]:
# is_iceberg; ids
y = [row['is_iceberg'] for row in train_data]
ids = [row['id'] for row in train_data]

In [90]:
y[:3], ids[:3]

([0, 0, 1], ['dfd5f913', 'e25388fd', '58b2aaa0'])

In [100]:
((val_ids,trn_ids),(val_data_array, trn_data_array),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(ids), train_bands, np.array(y))

In [101]:
len(val_ids), len(val_data_array), len(val_y)

(320, 320, 320)

In [102]:
len(trn_ids), len(trn_data_array), len(trn_y)

(1284, 1284, 1284)

In [103]:
train_array = (trn_data_array, trn_y)
val_array   = (val_data_array, val_y)

In [106]:
sz = 75
bs = 32
f_model = resnet34
PATH = 'data/'
tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
data = ImageClassifierData.from_arrays(PATH, train_array, val_array, tfms=tfms, bs=bs, 
                                       num_workers=8, classes=['0','1'], test=None)

testing `split_by_idx`

In [ ]:
def split_by_idx(idxs, *a):
    mask = np.zeros(len(a[0]),dtype=bool)
    mask[np.array(idxs)] = True
    return [(o[mask],o[~mask]) for o in a]

testing `... for i in X` where `X` is multiple arrays

In [92]:
[i for i in ([1,2,3],['a','b','c'],[0.5,0.7,0.9])]

[[1, 2, 3], ['a', 'b', 'c'], [0.5, 0.7, 0.9]]

In [107]:
learner = ConvLearner.pretrained(resnet34, data)

In [109]:
learner.summary

<bound method ConvLearner.summary of Sequential(
  (0): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

This notebook slightly cleaned up and its validation-splitting method tested against that in "new_attempt.ipynb" in "prelim_01_21JAN2018".